In [2]:
import pandas as pd

# 국세청 승인자료 면세파트 만들기

In [27]:
data = pd.read_excel('국세청승인자료(달랏사이트매출).xlsx')

In [28]:
# 조건에 따라 실공급가액 및 면세 컬럼 계산 및 추가
data['실공급가액'] = data.apply(lambda x: x['부가세']*10 if x['부가세'] != 0 else 0, axis=1)
data['봉사료'] = data.apply(lambda x: x['공급가액'] - x['실공급가액'] if x['부가세'] != 0 else x['공급가액'], axis=1)

In [29]:
# 면세가 음수인 경우, 해당 금액을 실공급가액에 더하고 면세를 0으로 업데이트
data['실공급가액'] = data.apply(lambda x: x['실공급가액'] + x['봉사료'] if x['봉사료'] < 0 else x['실공급가액'], axis=1)
data['봉사료'] = data.apply(lambda x: 0 if x['봉사료'] < 0 else x['봉사료'], axis=1)

In [30]:
data['실공급가액'] += data['봉사료'] % 10
data['봉사료'] -= data['봉사료'] % 10
data['공급가액'] = data['실공급가액']
data.drop('실공급가액', axis=1, inplace=True)
data.to_excel('국세청승인자료(달랏사이트매출)_면세금액정리.xlsx', index=False)

In [31]:
# 파일 경로
file_path_tax_approval = '국세청승인자료(달랏사이트매출)_면세금액정리.xlsx'
file_path_credit_card = '신용·체크카드_결제내역_2024.04.01-2024.05.01.xlsx'
file_path_cash_receipt = '현금영수증_2024.04.01-2024.05.01.xlsx'

# 각 파일 읽기
df_tax_approval = pd.read_excel(file_path_tax_approval)
df_credit_card = pd.read_excel(file_path_credit_card)
df_cash_receipt = pd.read_excel(file_path_cash_receipt)

In [32]:
df_credit_card.info()
df_tax_approval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   상점아이디(MID)  102 non-null    object 
 1   결제일시        102 non-null    object 
 2   취소일시        11 non-null     object 
 3   주문번호        102 non-null    object 
 4   결제상태        102 non-null    object 
 5   구매자명        102 non-null    object 
 6   결제액         97 non-null     float64
 7   취소액         102 non-null    float64
 8   부가세         102 non-null    float64
 9   공급가액        102 non-null    float64
 10  면세가액        102 non-null    float64
 11  결제수단        102 non-null    object 
 12  결제기관        102 non-null    object 
 13  구매상품        102 non-null    object 
 14  안내메시지       102 non-null    object 
 15  승인번호        102 non-null    float64
 16  취소자         11 non-null     object 
dtypes: float64(6), object(11)
memory usage: 13.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420 entries

In [33]:
# 국세청 데이터와 신용카드 결제 내역 데이터를 합치는 과정
# df_tax_approval은 국세청 데이터, merged_data는 신용카드 결제 내역 데이터를 나타냅니다.

# 새로운 데이터프레임 생성을 위해 필요한 컬럼만 df_2에서 선택
merged_data = df_credit_card[['결제일시', '공급가액', '부가세','면세가액', '결제액', '승인번호', '결제수단','주문번호']].copy()

# 컬럼 이름을 df_1과 일치시키기 위해 변경
merged_data.rename(columns={
    '결제일시': '매출일시',
    '결제액': '총금액',
    '결제수단': '거래구분',
    '면세가액': '봉사료'
}, inplace=True)

# 봉사료, 신분확인 뒷4자리, 용도구분, 비고 컬럼 추가 (값은 비워둠)
merged_data['신분확인뒷4자리'] = None  # 신분확인 뒷4자리는 비워둡니다.
merged_data['용도구분'] = None  # 용도구분은 비워둡니다.
merged_data['비고'] = None  # 비고는 비워둡니다.

# df_1 (국세청 데이터)과 합치기
final_df = pd.concat([df_tax_approval, merged_data], ignore_index=True)

# 최종 데이터프레임 확인
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1523 entries, 0 to 1522
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   발행구분      1419 non-null   object 
 1   매출일시      1521 non-null   object 
 2   공급가액      1521 non-null   float64
 3   부가세       1521 non-null   float64
 4   봉사료       1521 non-null   float64
 5   총금액       1517 non-null   float64
 6   승인번호      1521 non-null   float64
 7   신분확인뒷4자리  1419 non-null   float64
 8   거래구분      1521 non-null   object 
 9   용도구분      1419 non-null   object 
 10  비고        1419 non-null   object 
 11  주문번호      102 non-null    object 
dtypes: float64(6), object(6)
memory usage: 142.9+ KB


In [34]:
# 승인번호를 기준으로 신용체크카드 결제내역과 현금영수증 내역의 주문번호를 추출
cash_receipt_order_numbers = df_cash_receipt[['승인번호', '주문번호']].drop_duplicates(subset='승인번호')

# 국세청 승인자료에 주문번호 컬럼 추가하기
Ordernumber_merged = final_df.merge(cash_receipt_order_numbers, on='승인번호', how='left')

In [35]:
# '주문번호_x' 컬럼의 NaN 값을 '주문번호_y' 값으로 채우고, '주문번호'라는 새 컬럼에 저장
Ordernumber_merged['주문번호'] = Ordernumber_merged['주문번호_x'].fillna(Ordernumber_merged['주문번호_y'])

# 이제 '주문번호_x'와 '주문번호_y' 컬럼은 필요 없으므로 삭제
Ordernumber_merged.drop(columns=['주문번호_x', '주문번호_y'], inplace=True)

In [36]:
Ordernumber_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1523 entries, 0 to 1522
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   발행구분      1419 non-null   object 
 1   매출일시      1521 non-null   object 
 2   공급가액      1521 non-null   float64
 3   부가세       1521 non-null   float64
 4   봉사료       1521 non-null   float64
 5   총금액       1517 non-null   float64
 6   승인번호      1521 non-null   float64
 7   신분확인뒷4자리  1419 non-null   float64
 8   거래구분      1521 non-null   object 
 9   용도구분      1419 non-null   object 
 10  비고        1419 non-null   object 
 11  주문번호      1521 non-null   object 
dtypes: float64(6), object(6)
memory usage: 154.7+ KB


In [37]:
# 주문번호에서 '-'와 '-' 뒤의 값을 삭제
Ordernumber_merged['주문번호'] = Ordernumber_merged['주문번호'].str.split('-').str[0]

In [38]:
# 결과를 새 파일로 저장
output_file_path = '국세청승인자료_주문번호추가.xlsx'
Ordernumber_merged.to_excel(output_file_path, index=False)

# 저장된 파일 경로 반환
output_file_path

'국세청승인자료_주문번호추가.xlsx'

In [39]:
Ordernumber_merged

,발행구분,매출일시,공급가액,부가세,봉사료,총금액,승인번호,신분확인뒷4자리,거래구분,용도구분,비고,주문번호
0,사업자,2024-04-30 23:25:41,37555.0,3755.0,6390.0,47700.0,161032234.0,1234.0,승인거래,소비자소득공제용,일반거래,000002933
1,사업자,2024-04-30 20:31:59,146091.0,14609.0,75800.0,236500.0,160873755.0,1234.0,승인거래,소비자소득공제용,일반거래,000002932
2,사업자,2024-04-30 20:31:10,31182.0,3118.0,19600.0,53900.0,160874109.0,1234.0,승인거래,소비자소득공제용,일반거래,000002931
3,사업자,2024-04-30 18:33:44,19273.0,1927.0,16800.0,38000.0,160756462.0,1234.0,승인거래,소비자소득공제용,일반거래,000002922
4,사업자,2024-04-30 17:09:30,7636.0,764.0,0.0,8400.0,160668661.0,1234.0,승인거래,소비자소득공제용,일반거래,000002919
...,...,...,...,...,...,...,...,...,...,...,...,...
1518,NaN,2024-04-09 19:01:26,23273.0,2327.0,0.0,25600.0,75068369.0,NaN,일반,None,None,3000000570
1519,NaN,2024-04-01 09:29:46,20455.0,2045.0,0.0,22500.0,16703300.0,NaN,일반,None,None,000001546
1520,NaN,2024-04-04 14:57:02,-38182.0,-3818.0,0.0,NaN,479460.0,NaN,일반,None,None,3000000526
1521,NaN,2024-03-27 14:35:41,-3636.0,-364.0,-42000.0,NaN,59588609.0,NaN,일반,None,None,3000000442


# 주문번호 붙이기 완성 이제 고객명 붙이기

In [40]:
# 데이터 파일 로드
dalat_sales = pd.read_excel('4월 달랏매출.xlsx')
tax_approval_data = pd.read_excel('국세청승인자료_주문번호추가.xlsx')

# 'Order No'와 '주문번호'를 기준으로 두 데이터 프레임을 합친다
merged_data = pd.merge(tax_approval_data, dalat_sales[['Order No', '고객 이름','operator']], left_on='주문번호', right_on='Order No', how='left')

# 중복된 행 제거
final_data = merged_data.drop_duplicates()

# 필요한 컬럼만 선택하여 최종 데이터 프레임 생성
final_columns = ['발행구분', '매출일시', '공급가액', '부가세', '봉사료', '총금액', '승인번호', '신분확인뒷4자리', '거래구분', '용도구분', '비고', '주문번호', '고객 이름','operator']
final_data = final_data[final_columns]

# 최종 데이터를 Excel 파일로 저장
final_data_path = 'final_merged_data.xlsx'
final_data.to_excel(final_data_path, index=False)

이후에 나온 엑셀 데이터는 현금영수증 + 신용카드임. 
1. 신용카드에도 마이너스 금액을 총금액으로 옮겨주어야 함
2. 여신 금액 가져와서 더 붙여주기

# 노가다해서 나온 결과물 groupby 해주기

In [163]:
import openpyxl

In [170]:
# 엑셀 파일 경로
file_path = '달랏 3월 원장.xlsx'

# 첫 번째 시트의 데이터를 읽어옴
df = pd.read_excel(file_path, sheet_name=0)

# '고객 이름'과 '비고'를 기준으로 그룹화하고, 해당 그룹별로 '공급가액', '부가세', '봉사료', '총금액'의 합계 계산
grouped_by_customer_remark = df.groupby(['고객 이름', '비고'])[['공급가액', '부가세', '봉사료', '총금액']].sum().reset_index()

In [171]:
# 최종 데이터를 Excel 파일로 저장
grouped_path = '고객별 데이터.xlsx'
grouped_by_customer_remark.to_excel(grouped_path, index=False)